<a href="https://colab.research.google.com/github/vickytomar2/MovieRecommendation/blob/master/MovieDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# Import libraries
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reading ratings file
ratings = pd.read_csv('ratings.csv', sep='\t', encoding='latin-1', 
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['user_id'].drop_duplicates().max()
max_movieid = ratings['movie_id'].drop_duplicates().max()

# Reading ratings file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', 
                    usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading ratings file
movies = pd.read_csv('movies.csv', sep='\t', encoding='latin-1', 
                     usecols=['movie_id', 'title', 'genres'])

In [2]:
ratings.head()

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


In [3]:
movies.head()


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,user_id,gender,zipcode,age_desc,occ_desc
0,1,F,48067,Under 18,K-12 student
1,2,M,70072,56+,self-employed
2,3,M,55117,25-34,scientist
3,4,M,02460,45-49,executive/managerial
4,5,M,55455,25-34,writer


In [5]:
# Create training set
shuffled_ratings = ratings.sample(frac=1.)

# Shuffling users
Users = shuffled_ratings['user_emb_id'].values
print('Users:', Users, ', shape =', Users.shape)

# Shuffling movies
Movies = shuffled_ratings['movie_emb_id'].values
print('Movies:', Movies, ', shape =', Movies.shape)

# Shuffling ratings
Ratings = shuffled_ratings['rating'].values
print('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [3299 2125 3692 ... 3714 1067 5521] , shape = (1000209,)
Movies: [2528 3703 2119 ... 1096 2699 2468] , shape = (1000209,)
Ratings: [5 2 3 ... 5 4 4] , shape = (1000209,)


In [0]:
# Define constants
K_FACTORS = 100 # The number of dimensional embeddings for movies and users
TEST_USER = 1000 # A random test user (user_id = 2000)

In [27]:
pip install keras==2.1.2

     |████████████████████████████████| 307kB 9.0MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [8]:
from keras.layers import Embedding, Reshape, Merge
from keras.models import Sequential
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.


In [9]:
left = Sequential()
left.add(Embedding(max_userid, K_FACTORS,input_length=1))
left.add(Reshape((K_FACTORS,)))

right = Sequential()
right.add(Embedding(max_movieid,K_FACTORS, input_length=1))
right.add(Reshape((K_FACTORS,)))

model = Sequential()

model.add(Merge([left, right], mode='dot'))
model.compile(loss='mse', optimizer='adamax')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()


In [11]:
callbacks = [EarlyStopping('val_loss', patience=2),ModelCheckpoint('movie_weights.h5', save_best_only=True)]
model.fit([Users, Movies], Ratings, nb_epoch=50,validation_split=.1, callbacks=callbacks)

Train on 900188 samples, validate on 100021 samples
Epoch 1/50
   480/900188 [..............................] - ETA: 4:52 - loss: 14.4833

/usr/local/lib/python3.6/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


900188/900188 [==============================] - 220s 244us/step - loss: 7.3357 - val_loss: 2.0588
Epoch 2/50
900188/900188 [==============================] - 218s 242us/step - loss: 1.4040 - val_loss: 1.1071
Epoch 3/50
900188/900188 [==============================] - 218s 243us/step - loss: 0.9831 - val_loss: 0.9316
Epoch 4/50
900188/900188 [==============================] - 222s 246us/step - loss: 0.8768 - val_loss: 0.8690
Epoch 5/50
900188/900188 [==============================] - 225s 250us/step - loss: 0.8280 - val_loss: 0.8366
Epoch 6/50
900188/900188 [==============================] - 219s 243us/step - loss: 0.7968 - val_loss: 0.8161
Epoch 7/50
900188/900188 [==============================] - 209s 233us/step - loss: 0.7726 - val_loss: 0.8019
Epoch 8/50
900188/900188 [==============================] - 209s 232us/step - loss: 0.7515 - val_loss: 0.7912
Epoch 9/50
900188/900188 [==============================] - 207s 230us/step - loss: 0.7311 - val_loss: 0.7799
Epoch 10/50
900188/90

In [14]:

min_rmse = math.sqrt(0.7487)
print(min_rmse)

0.8652745229116595


In [0]:
import numpy as np
from keras.layers import Embedding, Reshape, Merge
from keras.models import Sequential

class CFModel(Sequential):

    # The constructor for the class
    def __init__(self, n_users, m_items, k_factors, **kwargs):
        # P is the embedding layer that creates an User by latent factors matrix.
        # If the intput is a user_id, P returns the latent factor vector for that user.
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1))
        P.add(Reshape((k_factors,)))

        # Q is the embedding layer that creates a Movie by latent factors matrix.
        # If the input is a movie_id, Q returns the latent factor vector for that movie.
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1))
        Q.add(Reshape((k_factors,)))

        super(CFModel, self).__init__(**kwargs)
        
        # The Merge layer takes the dot product of user and movie latent factor vectors to return the corresponding rating.
        self.add(Merge([P, Q], mode='dot', dot_axes=1))

    # The rate function to predict user's rating of unrated items
    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]

In [16]:
trained_model = CFModel(max_userid, max_movieid, K_FACTORS)
# Load weights
trained_model.load_weights('movie_weights.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [17]:
users[users['user_id'] == TEST_USER]

,user_id,gender,zipcode,age_desc,occ_desc
999,1000,F,90027,25-34,doctor/health care


In [0]:
# Function to predict the ratings given User ID and Movie ID
def predict_rating(user_id, movie_id):
    return trained_model.rate(user_id - 1, movie_id - 1)

In [19]:
user_ratings = ratings[ratings['user_id'] == TEST_USER][['user_id', 'movie_id', 'rating']]
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movie_id', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(20)

,user_id,movie_id,rating,prediction,title,genres
0,1000,3405,5,4.078745,"Night to Remember, A (1958)",Action|Drama
1,1000,3068,5,4.151642,"Verdict, The (1982)",Drama
2,1000,3418,5,4.194287,Thelma & Louise (1991),Action|Drama
3,1000,260,5,4.772050,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
4,1000,1023,5,4.348955,Winnie the Pooh and the Blustery Day (1968),Animation|Children's
5,1000,1104,5,4.432540,"Streetcar Named Desire, A (1951)",Drama
6,1000,2857,5,4.022496,Yellow Submarine (1968),Animation|Musical
7,1000,457,5,4.721675,"Fugitive, The (1993)",Action|Thriller
8,1000,318,5,5.077771,"Shawshank Redemption, The (1994)",Drama
9,1000,2571,5,4.985937,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [20]:
recommendations = ratings[ratings['movie_id'].isin(user_ratings['movie_id']) == False][['movie_id']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['movie_id']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movie_id',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(20)

,movie_id,prediction,title,genres
0,2324,4.966138,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama
1,2905,4.959520,Sanjuro (1962),Action|Adventure
2,2028,4.896996,Saving Private Ryan (1998),Action|Drama|War
3,2396,4.848960,Shakespeare in Love (1998),Comedy|Romance
4,3022,4.836109,"General, The (1927)",Comedy
5,668,4.811348,Pather Panchali (1955),Drama
6,2329,4.795425,American History X (1998),Drama
7,1111,4.793384,Microcosmos (Microcosmos: Le peuple de l'herbe...,Documentary
8,3030,4.792310,Yojimbo (1961),Comedy|Drama|Western
9,1148,4.779068,"Wrong Trousers, The (1993)",Animation|Comedy
